In [1]:
import fwiVis.fwiVis as fv
import s3fs
s3 = s3fs.S3FileSystem(anon=False)
from math import cos, asin, sqrt
import re

import numpy as np
import geopandas as gpd
import pandas as pd
from matplotlib import pyplot as plt
import os
import rioxarray as rio
import xarray as xr
import rasterio
import glob
from geocube.api.core import make_geocube
from shapely.errors import ShapelyDeprecationWarning
from shapely.geometry import Point
import warnings
import folium
import datetime
import time
from folium import plugins
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 

## Load up helper objects

files = s3.ls("s3://veda-data-store-staging/EIS/other/station-FWI/20170101.20221108.hrlyInterp/FWI/") # This will update through time. Not nessisary now because focus on 2019
st_id_map = pd.read_csv("/projects/my-public-bucket/fire_weather_vis/ref_data/isd-history.csv") ## For mappting ID's to lat lon

# Helper Variable
stations = fv.st_avail(files, st_id_map = st_id_map, path_s3 ="veda-data-store-staging/EIS/other/station-FWI/20170101.20221108.hrlyInterp/FWI/")
stations
st_dict = stations[['Lat', 'Lon']].to_dict('records')

Looking for availible stations atveda-data-store-staging/EIS/other/station-FWI/20170101.20221108.hrlyInterp/FWI/


Looking for files in ['/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/Thomas/2018/Largefire/F0_20180101AM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/Thomas/2018/Largefire/F0_20180101PM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/Thomas/2018/Largefire/F0_20180102AM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/Thomas/2018/Largefire/F0_20180102PM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/Thomas/2018/Largefire/F0_20180103AM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/Thomas/2018/Largefire/F0_20180103PM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/Thomas/2018/Largefire/F0_20180104AM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/Thomas/2018/Largefire/F0_20180104PM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/Thomas/2018/Largefire/F0_20180105AM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/Thomas/2018/Larg

/projects/my-public-bucket/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:371: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lon'] = gdf.centroid.x
/projects/my-public-bucket/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:372: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lat'] = gdf.centroid.y


,n_pixels,n_newpixels,farea,fperim,flinelen,duration,pixden,meanFRP,t,geometry,lon,lat
0,1929,480,376.318515,94.081343,44.265590,0.5,5.125977,75.582333,2017-12-05 12:00:00,"POLYGON ((-119.27724 34.27983, -119.27631 34.2...",-119.145970,34.368039
1,1449,1449,278.155930,80.526373,76.432527,0.0,5.209308,19.391629,2017-12-05 00:00:00,"POLYGON ((-119.27726 34.27967, -119.27724 34.2...",-119.129636,34.359356
2,3816,678,582.770896,159.018885,71.759202,2.0,6.548028,28.824676,2017-12-07 00:00:00,"POLYGON ((-119.27621 34.28519, -119.27613 34.2...",-119.159459,34.390847
3,4125,309,620.655298,174.391696,56.248701,2.5,6.646201,56.477799,2017-12-07 12:00:00,"MULTIPOLYGON (((-119.27591 34.28564, -119.2757...",-119.162605,34.395809
4,3138,277,485.849508,118.643452,28.902547,1.5,6.458790,82.096390,2017-12-06 12:00:00,"POLYGON ((-119.27627 34.28503, -119.27621 34.2...",-119.152853,34.377847
...,...,...,...,...,...,...,...,...,...,...,...,...
90,9135,263,1286.500534,200.481628,43.217869,11.5,7.100658,136.063612,2017-12-16 12:00:00,"POLYGON ((-119.29366 34.30729, -119.29371 34.3...",-119.249662,34.435689
91,9161,26,1288.101669,202.501053,8.739923,12.0,7.112016,2.426923,2017-12-17 00:00:00,"POLYGON ((-119.29371 34.30728, -119.29378 34.3...",-119.250072,34.435741
92,9235,74,1292.792093,202.346343,10.435023,12.5,7.143453,42.147297,2017-12-17 12:00:00,"POLYGON ((-119.29378 34.30727, -119.29383 34.3...",-119.249706,34.436153
93,9253,18,1293.248522,202.203816,1.893177,13.0,7.154851,11.685556,2017-12-18 00:00:00,"POLYGON ((-119.29383 34.30726, -119.29387 34.3...",-119.249656,34.436194


In [5]:
#caldor_lat = 38.586
#caldor_lon = -120.537833


foi = [{"Lat":38.586 , 
        "Lon":-120.537833 
    
}]
foi = pd.DataFrame(foi)
foi["Lat"]

0    38.586
Name: Lat, dtype: float64

In [25]:
st_cls = fv.closest(st_dict, foi)

print(st_cls)

## Get station Data
st = fv.get_st(lat = st_cls["Lat"], lon = st_cls["Lon"], stations = stations)

#gdf = gdf.sort_values(by = ['t']) ## Need to sort or timeseries jumps around
#gdf['t'] = gdf['t'].astype('datetime64[ns]')

#full_fr = fv.fr_st_merge(gdf, st, sub = True)
#full_fr
st

## Subset to relvatn dates (from CAL FIRE)

# Start 08/14/21 6:54 PM,  End 10/21/21 8:18AM

st = st[st.time > "2021-08-07 12:00:00"]

st = st[st.time < "2021-10-21 12:00:00"]
st

## Export
st.to_csv(path_or_buf= "/projects/shared-buckets/ashiklom/CALDOR_sbs/Caldor_fwi.csv") # 

The closest station is 23.48698806042275 km away.
{'Lat': 38.717, 'Lon': -120.75}


/projects/my-public-bucket/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:127: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  dat = pd.read_csv(("s3://" + st.File_path.iloc[0]), index_col = False)
/projects/my-public-bucket/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:127: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  dat = pd.read_csv(("s3://" + st.File_path.iloc[0]), index_col = False)
